# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
J. Shi  ->  J. Shi  |  ['J. Shi']
J. Liu  ->  J. Liu  |  ['J. Liu']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']


M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
J. Liu  ->  J. Liu  |  ['J. Liu']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
S. Stuber  ->  S. Stuber  |  ['S. Stuber']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']


Arxiv has 107 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.03424


extracting tarball to tmp_2407.03424... done.
Retrieving document from  https://arxiv.org/e-print/2407.03445


extracting tarball to tmp_2407.03445...

 done.
Retrieving document from  https://arxiv.org/e-print/2407.03520
extracting tarball to tmp_2407.03520... done.


...

extracting tarball to tmp_2407.04220... done.
Retrieving document from  https://arxiv.org/e-print/2407.04531


extracting tarball to tmp_2407.04531...

 done.


E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
S. Stuber  ->  S. Stuber  |  ['S. Stuber']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']


Found 64 bibliographic references in tmp_2407.04531/clean_version.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.03520-b31b1b.svg)](https://arxiv.org/abs/2407.03520) | **Planet Formation and Disk Chemistry: Dust and Gas Evolution during Planet Formation**  |
|| <mark>G. Perotti</mark>, et al. |
|*Appeared on*| *2024-07-08*|
|*Comments*| *9 pages, 5 figures, Chapter IV of the National Congress of (proto-)Planetary Astrochemistry (CNAP II) Proceedings*|
|**Abstract**|            Over the past decade, progress in observational capabilities, combined with theoretical advancements, have transformed our comprehension of the physics and chemistry during planet formation. Despite these important steps forward, open questions persist on the chemical and physical evolution of solids in their journey from the collapsing molecular cores to disks and planetary bodies. This chapter is a repository of such burning questions. It has the ambition to identify the most promising avenues for future research based on current observational and modeling opportunities.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.04094-b31b1b.svg)](https://arxiv.org/abs/2407.04094) | **Multi-wavelength properties of three new radio-powerful $z\sim5.6$ QSOs discovered from RACS**  |
|| L. Ighina, et al. -- incl., <mark>S. Belladitta</mark> |
|*Appeared on*| *2024-07-08*|
|*Comments*| *Submitted to A&A on July 4th*|
|**Abstract**|            We present a multi-wavelength study on three new $z\sim5.6$ quasi stellar objects (QSOs) selected based on their radio and optical/near-infrared properties in publicly available surveys and then identified with dedicated spectroscopic observations. These are among the radio-brightest QSOs currently known at $z>5.5$, having $\rm R=S_{\rm 5GHz}/S_{\rm 4400A}>100$. In this work we present their identification and we also discuss their multi-wavelength properties (from the radio to the X-ray band) based on the detection in public surveys as well as dedicated radio and X-ray observations. The three sources present a wide range of properties, in terms of relative intensity and spectral shape, highlighting the importance of multi-wavelength observations in order to accurately characterise these high-$z$ objects. In particular, from our analysis we found one source, at $z=5.61$, that presents clear blazars properties (strong and flat radio and X-ray emission), making it one of the most distant currently known in this class. Moreover, from the fit of the optical/near-infrared photometric measurements with an accretion disc model, as well as the analysis of the CIV broad emission line in one case, we were able to estimate the mass and the accretion rate of the central black holes in these systems, finding $\rm M_{\rm BH}\sim1-10\times10^9$ M$_\odot$ accreting at a rate $\lambda_{\rm Edd}\sim0.1-0.2$. With this work we increase the number of very-high redshift radio-powerful QSOs characterised with multi-wavelength observations, essential in order to understand the evolution and the properties of this still poorly constrained class of sources.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.04531-b31b1b.svg)](https://arxiv.org/abs/2407.04531) | **Neutral atomic and molecular gas dynamics in the nearby spiral galaxies NGC 1512, NGC 4535, and NGC 7496**  |
|| S. Laudage, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>S. Stuber</mark>, <mark>J. Neumann</mark> |
|*Appeared on*| *2024-07-08*|
|*Comments*| *accepted for publication in A&A; 13 pages, 9 Figures (+2 appendix pages)*|
|**Abstract**|            Neutral atomic gas (HI) effectively traces galactic dynamics across mid to large galactocentric radii. However, its limitations in observing small-scale changes within the central few kiloparsecs, coupled with the often observed HI deficit in galactic centers, necessitates using molecular gas emission as a preferred tracer in these regions. Understanding the dynamics of both neutral atomic and molecular gas is crucial for a more complete understanding of how galaxies evolve, funnel gas from the outer disk into their central parts, and eventually form stars. In this work we aim to quantify the dynamics of both, the neutral atomic and molecular gas, in the nearby spiral galaxies NGC 1512, NGC 4535, and NGC 7496 using new MeerKAT-HI observations together with ALMA CO (2-1) observations from the PHANGS collaboration. We use the analysis tool 3D-Barolo to fit tilted ring models to the HI and CO observations. A combined approach of using the HI to constrain the true disk orientation parameters before applying these to the CO datasets is tested. This paper sets expectations for the results of the upcoming high-resolution HI coverage of many galaxies in the PHANGS-ALMA sample using MeerKAT or VLA, to establish a robust methodology for characterizing galaxy orientations and deriving dynamics from combining new HI with existing CO data.         |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.03713-b31b1b.svg)](https://arxiv.org/abs/2407.03713) | **Compositions of the Hercules-Aquila Cloud and Virgo Over-density**  |
|| D. Ye, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2024-07-08*|
|*Comments*| *11 pages, 9 figures, accepted for publication in MNRAS*|
|**Abstract**|            Based on a sample of K giant from Large sky Area Multi-Object fiber Spectroscopic Telescope (LAMOST) Data Release 8 and a sample of RR Lyrae (RRL) from \textit{Gaia} Data Release 3, we investigate the compositions of the Hercules-Aquila Cloud (HAC) and Virgo Over-density (VOD) and their collective contribution to the tilt and triaxiality of the stellar halo ($r\,\textless\,40\,{\rm kpc}$) as well as two breaks at $\approx15\,{\rm kpc}$ and 30\,kpc. We apply the Gaussian mixture model (GMM) to divide the stellar halo into the isotropic component and the radially biased anisotropic component, namely Gaia-Sausage-Enceladus (GSE), and find that both HAC and VOD are dominated by the GSE debris stars with weights of $0.67^{+0.09}_{-0.07}$ and $0.57^{+0.07}_{-0.06}$, respectively. In addition, using the K giants with orbital parameters, we identify the member stars of known substructures, including GSE, Sagittarius (Sgr), Helmi Streams, Sequoia, Thamnos, Pontus, Wukong, and Metal-weak Thick Disk (MWTD), to probe the compositions of low-eccentricity stars in the HAC and VOD regions. In density fittings of the RRL sample, we note that the absence of HAC and VOD has a weak effect on the shape of halo. Finally, we find that the radially biased anisotropic halo contributes majorly to the stellar halo that can be modelled with a tilted triaxial ellipsoid and a doubly broken power law with breaking radii at $18.08^{+2.04}_{-3.22}\,{\rm kpc}$ and $33.03^{+1.30}_{-1.21}\,{\rm kpc}$. This has important significance for understanding the status of large diffuse over-densities in the Milky Way.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.03721-b31b1b.svg)](https://arxiv.org/abs/2407.03721) | **Discovery of a dusty yellow supergiant progenitor for the Type IIb SN 2017gkk**  |
|| Z. Niu, N.-C. Sun, <mark>J. Liu</mark> |
|*Appeared on*| *2024-07-08*|
|*Comments*| *6 pages, 3 figures. ApJL accepted. Comments are welcome*|
|**Abstract**|            Type IIb supernovae are important subclass of stripped-envelope supernovae (SNe), which show H lines only at early times. Their progenitors are believed to contain a low-mass H envelope before explosion. This work reports the discovery of a progenitor candidate in pre-explosion Hubble Space Telescope images for the Type IIb SN~2017gkk. With detailed analysis of its spectral energy distribution and local environment, we suggest that the progenitor is most likely a yellow supergiant with significant circumstellar extinction and has an initial mass of about 16 $M_\odot$, effective temperature log($T_{\rm eff}/K)=3.72\pm0.08$ and luminosity log($L/L_{\odot})=5.17\pm0.04$. This progenitor is not massive enough to strip envelope through stellar wind, and it supports an interacting binary progenitor channel and adds to the growing list of direct progenitor detections for Type~IIb SNe. Future late-time observations will confirm whether this progenitor candidate has disappeared and reveal the putative binary companion that has survived the explosion.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.04164-b31b1b.svg)](https://arxiv.org/abs/2407.04164) | **AGN STORM 2: VIII. Investigating the Narrow Absorption Lines in Mrk 817 Using HST-COS Observations**  |
|| M. Dehghanian, et al. -- incl., <mark>M. Sharma</mark> |
|*Appeared on*| *2024-07-08*|
|*Comments*| *18 pages, 12 Figures, Accepted for publication in ApJ*|
|**Abstract**|            We observed the Seyfert 1 galaxy Mrk817 during an intensive multi-wavelength reverberation mapping campaign for 16 months. Here, we examine the behavior of narrow UV absorption lines seen in HST/COS spectra, both during the campaign and in other epochs extending over 14 years. We conclude that while the narrow absorption outflow system (at -3750 km/s with FWHM=177 km/s) responds to the variations of the UV continuum as modified by the X-ray obscurer, its total column density (logNH =19.5 cm-2) did not change across all epochs. The adjusted ionization parameter (scaled with respect to the variations in the Hydrogen ionizing continuum flux) is log UH =-1.0. The outflow is located at a distance smaller than 38 parsecs from the central source, which implies a hydrogen density of nH > 3000 cm-3. The absorption outflow system only covers the continuum emission source and not the broad emission line region, which suggests that its transverse size is small (< 1e16 cm), with potential cloud geometries ranging from spherical to elongated along the line of sight.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.04220-b31b1b.svg)](https://arxiv.org/abs/2407.04220) | **Detecting the dark sector through scalar-induced gravitational waves**  |
|| X.-B. Sui, <mark>J. Liu</mark>, X.-Y. Yang, R.-G. Cai |
|*Appeared on*| *2024-07-08*|
|*Comments*| *13 pages, 4 figures*|
|**Abstract**|            We investigate the evolution of cosmological scalar perturbations in the case that the background radiation is weakly coupled to a light scalar field $\phi$. The light scalar $\phi$ is a homogeneous background field with a large initial value. In the radiation-dominated Universe, the coupling term introduces an effective mass to $\phi$ and the background ultra-relativistic particles. The oscillations of $\phi$ result in the periodic change of the equation of state parameter and the sound speed, which provides a novel mechanism to amplify subhorizon scalar perturbations through parametric resonance. The amplification of scalar perturbations leads to a stochastic gravitational-waves background~(SGWB) expected to be observed by multiband gravitational wave observers. The observation of the SGWB helps to determine the initial value of $\phi$ and the coupling strength of the interaction. This mechanism is generally applicable to the interactions that introduce an effective mass, and we take the interaction between $\phi$ and electrons as a concrete example to illustrate the result. We find that under the condition that the coupling coefficient $\lambda=10^{-16}$ and the initial value $\phi_i=10^{18}$ GeV, the resulting SGWB spectrum is expected to be observed by the future observers including LISA, $Taiji$, DECIGO and BBO.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.03445-b31b1b.svg)](https://arxiv.org/abs/2407.03445) | **Submillimeter and Mid-Infrared Variability of Young Stellar Objects in the M17SWex Intermediate-Mass Star-Forming Region**  |
|| G. Park, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-07-08*|
|*Comments*| *Accepted for Publication in The Astronomical Journal*|
|**Abstract**|            We present a comprehensive analysis of young stellar object (YSO) variability within the M17 Southwest Extension (M17 SWex), using 3.5 years of monitoring data from the JCMT Transient Survey at sub-millimeter (sub-mm) and 9 years from the NEOWISE mission at mid-infrared (mid-IR). Our study encompasses observations of 147 bright sub-mm peaks identified within our deep JCMT co-added map as well as 156 YSOs in NEOWISE W1 and 179 in W2 that were previously identified in Spitzer surveys. We find three robust sub-mm variables: two are candidate YSOs and one is a likely extragalactic source. At mid-IR wavelengths, our analysis reveals secular and stochastic variability in 47 YSOs, with the highest fraction of secular variability occurring at the earliest evolutionary stage. This is similar to what has previously been observed for low-mass YSO variability within the Gould Belt. However, we observe less overall variability in M17SWex at both the sub-mm and mid-IR. We suspect that this lower fraction is due to the greater distance to M17 SWex. Our findings showcase the utility of multi-wavelength observations to better capture the complex variability phenomena inherent to star formation processes and demonstrate the importance of years-long monitoring of a diverse selection of star-forming environments.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.03520/./Artboard_5.png', 'tmp_2407.03520/./LC_vertical_last.png', 'tmp_2407.03520/./lc_adapt.png']
copying  tmp_2407.03520/./Artboard_5.png to _build/html/
copying  tmp_2407.03520/./LC_vertical_last.png to _build/html/
copying  tmp_2407.03520/./lc_adapt.png to _build/html/
exported in  _build/html/2407.03520.md
    + _build/html/tmp_2407.03520/./Artboard_5.png
    + _build/html/tmp_2407.03520/./LC_vertical_last.png
    + _build/html/tmp_2407.03520/./lc_adapt.png
found figures ['tmp_2407.04094/./figures/SED_all.png', 'tmp_2407.04094/./figures/radio_spec_3bla.png', 'tmp_2407.04094/./figures/comparison_R_aox.png']
copying  tmp_2407.04094/./figures/SED_all.png to _build/html/
copying  tmp_2407.04094/./figures/radio_spec_3bla.png to _build/html/
copying  tmp_2407.04094/./figures/comparison_R_aox.png to _build/html/
exported in  _build/html/2407.04094.md
    + _build/html/tmp_2407.04094/./figures/SED_all.png
    + _build/html/tmp_2407.04094/./figures/radio_spec_3bla.

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\gp}[1]{\textcolor{teal}{\textbf{GP:} #1}}$
$\newcommand{\teff}{T\rm_{eff }}$
$\newcommand{\kms}{\mathrm{km s}^{-1}}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# Planet Formation and Disk Chemistry

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.03520-b31b1b.svg)](https://arxiv.org/abs/2407.03520)<mark>Appeared on: 2024-07-08</mark> -  _9 pages, 5 figures, Chapter IV of the National Congress of (proto-)Planetary Astrochemistry (CNAP II) Proceedings_

</div>
<div id="authors">

<mark>G. Perotti</mark>, et al.

</div>
<div id="abstract">

**Abstract:** Over the past decade, progress in observational capabilities, combined with theoretical advancements, have transformed our comprehension of the physics and chemistry during planet formation. Despite these important steps forward, open questions persist on the chemical and physical evolution of solids in their journey from the collapsing molecular cores to disks and planetary bodies. This chapter is a repository of such burning questions. It has the ambition to identify the most promising avenues for future research based on current observational and modeling opportunities. $\keywords{Planet formation, Protoplanetary disks, Circumstellar matter, Dust, Gas}$

</div>

<div id="div_fig1">

<img src="tmp_2407.03520/./Artboard_5.png" alt="Fig5" width="100%"/>

**Figure 5. -** Schematics of a planet-forming disk. The illustration highlights facilities used to probe different dust and gas disk reservoirs. Figure by A. Houge. (*fig:disk_sketch*)

</div>
<div id="div_fig2">

<img src="tmp_2407.03520/./LC_vertical_last.png" alt="Fig1" width="100%"/>

**Figure 1. -** Complementary JWST and ALMA observations the L1527 IRS protostar. _Top:_ JWST/NIRCam image highlighting the wide molecular outflow (PID: 2739). _Middle:_ ALMA $^{13}$CO contours showing a snapshot of gas moving at +0.5 km/s with respect to the central source (PID: 2019.1.00261.L). _Bottom:_ ALMA 1.3 mm dust emission contours probing the disk midplane (adapted from \citealt{2023ApJ...951...10V}). (*fig:L1527*)

</div>
<div id="div_fig3">

<img src="tmp_2407.03520/./lc_adapt.png" alt="Fig2" width="100%"/>

**Figure 2. -** Multi-wavelength dust opacity spectral index ($\beta_{0.88-3.4 mm}$) across the envelope of L1527, as a function of ALMA antennas baseline length or, equivalently, recovered physical scale (top x-axis). The probed scales go from 2000 au to 100 au. A gradient in dust properties across scales could be explained by dust growth to sub-millimeter sizes, variations in the chemical compositions of grains, or outwards transport of disk grains along outflows. Adapted from \citep{Cacciapuoti2023}. (*fig:LC23*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.03520"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Multi-wavelength properties of three new radio-powerful $z\sim5.6$ QSOs discovered from RACS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.04094-b31b1b.svg)](https://arxiv.org/abs/2407.04094)<mark>Appeared on: 2024-07-08</mark> -  _Submitted to A&A on July 4th_

</div>
<div id="authors">

L. Ighina, et al. -- incl., <mark>S. Belladitta</mark>

</div>
<div id="abstract">

**Abstract:** We present a multi-wavelength study on three new $z\sim5.6$ quasi stellar objects (QSOs) selected based on their radio and optical/near-infrared properties in publicly available surveys and then identified with dedicated spectroscopic observations. These are among the radio-brightest QSOs currently known at $z>5.5$ , having $\rm R=S_{\rm 5GHz}/S_{\rm 4400Å}>100$ . In this work we present their identification and we also discuss their multi-wavelength properties (from the radio to the X-ray band) based on the detection in public surveys as well as dedicated radio and X-ray observations. The three sources present a wide range of properties, in terms of relative intensity and spectral shape, highlighting the importance of multi-wavelength observations in order to accurately characterise these high- $z$ objects. In particular, from our analysis we found one source, at $z=5.61$ , that presents clear blazars properties (strong and flat radio and X-ray emission), making it one of the most distant currently known in this class. Moreover, from the fit of the optical/near-infrared photometric measurements with an accretion disc model, as well as the analysis of the CIV broad emission line in one case, we were able to estimate the mass and the accretion rate of the central black holes in these systems, finding $\rm M_{\rm BH}\sim1-10\times10^9$ M $_\odot$ accreting at a rate $\lambda_{\rm Edd}\sim0.1-0.2$ . With this work we increase the number of very-high redshift radio-powerful QSOs characterised with multi-wavelength observations, essential in order to understand the evolution and the properties of this still poorly constrained class of sources.

</div>

<div id="div_fig1">

<img src="tmp_2407.04094/./figures/SED_all.png" alt="Fig7" width="100%"/>

**Figure 7. -** Multi-wavelength rest-frame SED of the three new RL QSOs presented in this work. For each source we show the radio (red squares), optical/NIR (purple circles) and X-ray (orange diamonds) photometry discussed in the text. The black line is a QSO template from {\protect [Polletta, Tajer and Maraschi (2007)]()}, while the shaded yellow region represents frequencies affected by the hydrogen absorption from the inter-galactic medium (IGM). The dashed line is the best-fit AD model to the optical data-points (assuming $\eta\sim0.1$) together with the expected X-ray emission from the optical/UV--X-ray relation found in {\protect [ and Lusso (2016)]()}(assuming $\Gamma_{\rm X}=2$ and an exponential cut-off at 200 keV). The Corresponding shaded area in the X-rays shows the 1 and 2$\sigma$ scatter of the relation. The dotted black line at low frequency is the best-fit power law to the radio data, as described in the text. (*fig:SED*)

</div>
<div id="div_fig2">

<img src="tmp_2407.04094/./figures/radio_spec_3bla.png" alt="Fig4" width="100%"/>

**Figure 4. -** Radio spectra over the $\sim0.1-10$ GHz observed frequency range of the three sources described in this work. The majority of the data points are from public surveys (circles), while the 2.1, 5.5 and 9 GHz measurements are from dedicated, simultaneous observations with ATCA (squares). The dashed line in each panel indicate the best-fit model obtained for the different objects as described in the text. (*fig:radio_spec*)

</div>
<div id="div_fig3">

<img src="tmp_2407.04094/./figures/comparison_R_aox.png" alt="Fig5" width="100%"/>

**Figure 5. -** Radio loudness as a function of the $\tilde{\alpha}_{\rm ox}$ parameter for the objects presented in this work (blue circle, red square and green upper limit) as well as for a complete radio-selected sample of $z>4$ blazars ( (Caccianiga2019, [Ighina, Caccianiga and Moretti 2019]()) ) and $z>4$ radio QSOs discussed from the literature ( ([Caccianiga, Ighina and Moretti 2024]()) , black points, and from single studies, pink symbols, see text). The shaded grey regions at $\tilde{\alpha}_{\rm ox}\sim1.35$ and log(R) $\sim2$ divide the parameter space in two main regions: X-ray-, radio-bright objects (top-right corner, likely blazars) and X-ray-, radio-faint objects (bottom-left corner, likely hosting mis-aligned jets, i.e., classic RL QSOs). (*fig:R_aox*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.04094"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{  \hi       }{\ifmmode{\rm H} \textsc{i} \else H \textsc{i} \fi}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\sample}{NGC~1512, NGC~4535, and NGC~7496}$
$\newcommand{\orcid}[1]{\href{https://orcid.org/#1}{\includegraphics[width=10pt]{figs/orcid.pdf}}}$
$\newcommand{\Bonn}{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\NRAO}{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903, USA}$
$\newcommand{\OSU}{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\newcommand{ÇAP}{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\Princeton}{Department of Astrophysical Sciences, Princeton University, 4 Ivy Lane, Princeton, NJ 08544, USA}$
$\newcommand{\MPIA}{\affiliation{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}}$
$\newcommand{\HeidelbergARI}{\affiliation{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{\HeidelbergITA}{Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Stra{\ss}e 2,69120 Heidelberg, Germany}$
$\newcommand{\HeidelbergZWR}{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, 69120 Heidelberg, Germany}$
$\newcommand{\ESO}{\affiliation{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}}$
$\newcommand{\Alberta}{\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{\OAN}{Observatorio Astron{ó}mico Nacional (IGN), C/ Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\astron}{Netherlands Institute for Radio Astronomy (ASTRON),  Oude Hoogeveensedijk 4, 7991 PD Dwingeloo, Netherlands}$
$\newcommand{\kapeyn}{Kapteyn Astronomical Institute, University of Groningen, PO Box 800, 9700 AV Groningen, The Netherlands}$
$\newcommand{\UCT}{Department of Astronomy, University of Cape Town, Private Bag X3, 7701 Rondebosch, South Africa}$
$\newcommand{\MPE}{\affiliation{Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}}$
$\newcommand{\Conneticut}{Department of Physics, University of Connecticut, Storrs, CT, 06269, USA}$
$\newcommand{\ASIAA}{Institute of Astronomy and Astrophysics, Academia Sinica, Astronomy-Mathematics Building, No. 1, Sec. 4, Roosevelt Road, Taipei 10617, Taiwan}$
$\newcommand{\NRAOsoc}{National Radio Astronomy Observatory, 1003 Lopezville Road, Socorro, NM 87801, USA}$
$\newcommand{\Wyoming}{Department of Physics & Astronomy, University of Wyoming, Laramie, WY 82071}$
$\newcommand{\Tamkang}{Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan}$
$\newcommand{\SanDiego}{Center for Astrophysics and Space Sciences, University of California, San Diego, 9500 Gilman Drive MC0424, La Jolla, CA 92093, USA}$
$\newcommand{\Oxford}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\sectionautorefname}{Section}$</div>



<div id="title">

# Neutral atomic and molecular gas dynamics in the nearby spiral galaxies NGC 1512, NGC 4535, and NGC 7496

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.04531-b31b1b.svg)](https://arxiv.org/abs/2407.04531)<mark>Appeared on: 2024-07-08</mark> -  _accepted for publication in A&A; 13 pages, 9 Figures (+2 appendix pages)_

</div>
<div id="authors">

S. Laudage, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>S. Stuber</mark>, <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** Neutral atomic gas ( $\rm H i$ ) effectively traces galactic dynamics across mid to large galactocentric radii. However, its limitations in observing small-scale changes within the central few kiloparsecs, coupled with the often observed $\hi$ deficit in galactic centers, necessitates the use of molecular gas emission as a preferred tracer in these regions. Understanding the dynamics of both neutral atomic and molecular gas is crucial for a more complete understanding of how galaxies evolve, funnel gas from the outer disk into their central parts, and eventually form stars. In this work we aim to quantify the dynamics of both, the neutral atomic and molecular gas, in the nearby spiral galaxies NGC 1512, NGC 4535, and NGC 7496 using new MeerKAT $\hi$ observations together with ALMA CO (2-1) observations from the PHANGS collaboration. We use the analysis tool $\BB$ to fit tilted ring models to the $\hi$ and CO observations. A combined approach of using the $\hi$ to constrain the true disk orientation parameters before applying these to the CO datasets is tested. This paper sets expectations for the results of the upcoming high-resolution $\hi$ coverage of many galaxies in the PHANGS-ALMA sample using MeerKAT or VLA, to establish a robust methodology for characterizing galaxy orientations and deriving dynamics from combing new $\hi$ with existing CO data.

</div>

<div id="div_fig1">

<img src="tmp_2407.04531/./figs/Figure3.png" alt="Fig3" width="100%"/>

**Figure 3. -** _Top row:_ Final $\hi$ rotation curves (black circles) for NGC 1512 (left), NGC 4535 (middle) and NGC 7496 (right) overplotted with rotation curves derived by only considering the approaching (blue) and the receding (red) side of the galaxy. The bar radius is indicated with a dashed grey line and the distance of NGC 1510 is indicated with the green dashed line. _Bottom row:_ The panels show the adopted geometry in our fitting procedure for NGC 1512 (left), NGC 4535 (middle) and NGC 7496 (right) for the $\hi$ observations. The grey squares and circles represent the inclination $i$ and position angle $\phi$ results for each ring after the second fitting stage, respectively. The blue (inclination) and yellow (position angle) lines represent the regularization function that was used to smooth out the geometry in the final stage of the fitting procedure. (*fig:ResultsHI*)

</div>
<div id="div_fig2">

<img src="tmp_2407.04531/./figs/Figure1.png" alt="Fig1" width="100%"/>

**Figure 1. -** _Top row:_ Moment 1 maps from PHANGS MeerKAT $\hi$ observations of NGC 1512, NGC 4535, and NGC 7496 built with the strict masking procedure described in Section \ref{sec:Methods}.  The maps are overplotted with isovelocity-lines (grey) in intervals of \SI{30}{\kilo\meter\per\second}. _Middle row:_ Integrated intensity or Moment 0 maps from the MeerKAT $\hi$ observations built with the same masking procedure. The location of the companion galaxy NGC 1510 is indicated by the white circle. _Bottom row:_ Integrated intensity or Moment 0 maps of the three galaxies built from CO observations taken from PHANGS-ALMA. The extent of the CO maps is indicated in comparison to the $\hi$ maps. (*fig:moment1_HI*)

</div>
<div id="div_fig3">

<img src="tmp_2407.04531/./figs/PVs.png" alt="Fig2" width="100%"/>

**Figure 2. -** \centering_Top row:_ Line-of-sight velocities along the major axis position from the $\hi$ observations of NGC 1512 (left), NGC 4535 (middle), and NGC 7496 (right). _Bottom row:_ Line-of-sight velocities along the minor axis. The x-axis represents the offset of the galactic center and the y-axis represents the observed velocities with (left) and without (right) subtracting the systemic velocity of the galaxy. (*fig:PVs_MAJOR*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.04531"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

117  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
